In [1]:
from __future__ import print_function, absolute_import

import os 
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torchvision
import torchvision.datasets as datasets
import numpy as np

import sys

sys.path.append('..')

import scripts.utils
from scripts.utils.logger import Logger, savefig
from scripts.utils.evaluation import accuracy, AverageMeter, final_preds
from scripts.utils.misc import save_checkpoint, save_pred, adjust_learning_rate
from scripts.utils.osutils import mkdir_p, isfile, isdir, join
from scripts.utils.imutils import batch_with_heatmap,normalize_batch,im_to_numpy
from scripts.utils.transforms import fliplr, flip_back
import scripts.models as models
import scripts.datasets as datasets

from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr

class objectview(object):
    """Convert dict(or parameters of dict) to object view
    See also:
        - https://goodcode.io/articles/python-dict-object/
        - https://stackoverflow.com/questions/1305532/convert-python-dict-to-object
    >>> o = objectview({'a': 1, 'b': 2})
    >>> o.a, o.b
    (1, 2)
    >>> o = objectview(a=1, b=2)
    >>> o.a, o.b
    (1, 2)
    """
    def __init__(self, *args, **kwargs):
        d = dict(*args, **kwargs)
        self.__dict__ = d

        
methods = ('/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_can',
          '/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_dih256',
          '/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_dihrasc',
          '/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_radhnv3xgaussian',
          '/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_unet'
          )

dataset_name= 'val5k'
dataroot = '/home/oishii/Datasets/fivek/'

data_config  = objectview({'input_size':256,'normalized_input':False,'data_augumentation':False,'withseg':False})

val_loader = torch.utils.data.DataLoader(
        datasets.COCO(dataroot,dataset_name,config=data_config),
        batch_size=1, shuffle=False,
        num_workers=2, pin_memory=False)

for resume in methods:

    os.makedirs(os.path.join(resume,dataset_name,'input'),exist_ok=True)
    os.makedirs(os.path.join(resume,dataset_name,'output'),exist_ok=True)
    os.makedirs(os.path.join(resume,dataset_name,'mask'),exist_ok=True)
    os.makedirs(os.path.join(resume,dataset_name,'target'),exist_ok=True)
#     os.makedirs(os.path.join(resume,dataset_name,'all'),exist_ok=True)
#     if 'styleloss' in resume:
#         model = models.__dict__[resume.replace('xstyleloss','').split('_')[-1]]().cuda()
#     else:
    model = models.__dict__[resume.split('_')[-1]]().cuda()
    checkpoint = torch.load(resume+'/model_best.pth.tar')
    best_scc = checkpoint['best_acc']
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    
    MSE = AverageMeter()
    PSNR = AverageMeter()
    SSIM = AverageMeter()
    
    with torch.no_grad():
        for i, (inputs, target) in enumerate(val_loader):            
            inputs = inputs.cuda()
            target = target[0].cuda()
            output = model(inputs)
 
            outputnp = (im_to_numpy(output[0])*255).astype(np.uint8)
            targetnp = (im_to_numpy(target[0])*255).astype(np.uint8)
            inputsnp = (im_to_numpy(inputs[0,0:3])*255).astype(np.uint8)
            
            tmp_ssim = ssim(targetnp,outputnp,multichannel=True)
            tmp_psnr = psnr(targetnp,outputnp)
            tmp_mse = np.mean( (outputnp - targetnp) ** 2 )
            
            MSE.update(tmp_mse, inputs.size(0))
            PSNR.update(tmp_psnr, inputs.size(0))
            SSIM.update(tmp_ssim, inputs.size(0))
            
            torchvision.utils.save_image(inputs[0,3:4,:,:],os.path.join(resume,dataset_name,'mask','%s.png'%(i)),padding=0)
            torchvision.utils.save_image(target[0],os.path.join(resume,dataset_name,'target','%s.png'%(i)),padding=0)
            torchvision.utils.save_image(inputs[0,0:3,:,:],os.path.join(resume,dataset_name,'input','%s.png'%(i)),padding=0)
            torchvision.utils.save_image(output[0],os.path.join(resume,dataset_name,'output','%s.png'%(i)),padding=0)
#             torchvision.utils.save_image(torch.cat([inputs[:,0:3,:,:],target,output],dim=0),os.path.join(resume,dataset_name,'all','%s.png'%(i)))
            
    print("%s: MSE:%s, SSIM:%s, PSNR:%s"%(resume, MSE.avg, SSIM.avg, PSNR.avg))
            


total Dataset of val5k is :  2420


../scripts/utils/model_init.py:38: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 1.0, 0.02)
../scripts/utils/model_init.py:39: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_can: MSE:25.99098691283512, SSIM:0.9062486607292697, PSNR:23.718771400806354
/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_dih256: MSE:34.97740772552063, SSIM:0.862660646693442, PSNR:20.956739032792978
/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_dihrasc: MSE:27.153652462289365, SSIM:0.9259332316411045, PSNR:24.730374635275105
/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_radhnv3xgaussian: MSE:21.875379460442467, SSIM:0.9571092344930275, PSNR:28.909926169442468
/home/oishii/Documents/deep-harimonization-improved/psnr/adobe5k/1e3_bs16_256_5k_unet: MSE:23.212920585611336, SSIM:0.9491754482234419, PSNR:27.07506575555279
